In [1]:
# Importar librerias
from   bs4 import BeautifulSoup
import requests

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os

import re

# Variables globables
HEADER = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
DEFAULT_UTILS_VERBOSE = True

In [113]:
def get_http_response(url, headers=None, response_type='page', verbose=DEFAULT_UTILS_VERBOSE, debug=False, timeout=10, retry_attempts=3):
    """
    Obtiene la respuesta HTML de una URL.
    Puede aceptar headers personalizados; si no se proporcionan, utiliza unos por defecto.
    La función retorna el resultado HTML como un objeto BeautifulSoup o texto plano.

    Args:
        url (str): La URL de la página web.
        headers (dict, optional): Headers para la solicitud HTTP.
        response_type (str, optional): Tipo de respuesta ('page' para BeautifulSoup, 'text' para texto plano).
        verbose (bool, optional): Si es True, imprime información detallada.
        debug (bool, optional): Si es True, imprime la respuesta HTTP completa.
        timeout (int, optional): Tiempo máximo de espera para la solicitud HTTP en segundos.
        retry_attempts (int, optional): Número de intentos de reintentos en caso de fallo.

    Returns:
        BeautifulSoup object or str: Dependiendo de response_type, retorna un objeto BeautifulSoup o texto plano.

    Raises:
        ValueError: Si response_type no es 'page' o 'text'.
        RuntimeError: Si ocurre un error durante la solicitud HTTP.
    """
    # Validación de parámetros
    if not isinstance(url, str):
        raise ValueError("La URL debe ser una cadena de caracteres.")
    if headers is not None and not isinstance(headers, dict):
        raise ValueError("Headers debe ser un diccionario.")
    if response_type not in ['page', 'text']:
        raise ValueError("response_type debe ser 'page' o 'text'.")
    if not isinstance(verbose, bool):
        raise ValueError("verbose debe ser un valor booleano.")
    if not isinstance(debug, bool):
        raise ValueError("debug debe ser un valor booleano.")
    if not isinstance(timeout, (int, float)):
        raise ValueError("timeout debe ser un número.")
    if not isinstance(retry_attempts, int) or retry_attempts < 0:
        raise ValueError("El número de intentos de reintentos debe ser un entero no negativo.")

    # Definimos los headers por defecto
    if headers is None:
        headers = {
            'user-agent': HEADER
        }

    attempts = 0
    while attempts <= retry_attempts:
        try:
            # Realizamos una solicitud a la página web con timeout
            response = requests.get(url, headers=headers, timeout=timeout)

            # Debug: Imprimimos la respuesta completa si debug es True
            if debug:
                print(f'HTTP response: {response}')

            # Analizamos el contenido HTML de la página web utilizando BeautifulSoup
            page = BeautifulSoup(response.content, 'html.parser')

            # Verbose: Imprimimos información detallada si verbose es True
            if verbose:
                msg  = f'URL [{url}], '
                msg += f'HTTP status [{response.ok}], '
                msg += f'HTTP code [{response.status_code}]'
                print(msg)

            # Validación de la respuesta
            if response.ok:
                if response_type == 'text':
                    return response.text
                else:
                    return page
            else:
                if verbose:
                    msg  = f'URL [{url}], '
                    msg += f'HTTP status [{response.ok}], '
                    msg += f'HTTP code [{response.status_code}], '
                    msg += f'Message [ERROR! Ocurrió un error inesperado al cargar la URL seleccionada]'
                    print(msg)
                return None

        except requests.RequestException as e:
            if verbose:
                logger.error(f'ERROR! Ocurrió un error al realizar la solicitud HTTP para la URL [{url}]. Error: [{e}]')

            # Incrementamos el contador de intentos y esperamos antes de reintentar
            attempts += 1
            if attempts <= retry_attempts:
                time.sleep(1)  # Esperamos 1 segundo antes de realizar el siguiente intento

    # Si llegamos aquí, significa que todos los intentos de reintentos fallaron
    print(f"No se pudo obtener la respuesta HTTP para la URL [{url}] después de {retry_attempts} intentos.")
    return None

def calculate_average(price_string):
    """
    Calcula el promedio de dos precios en una cadena o devuelve el precio único si solo hay uno.
    
    Parámetros:
    price_string (str): Cadena que contiene uno o dos precios con la moneda.

    Retorna:
    str: El promedio de los precios o el precio único, formateado con la moneda.
    """
    # Reemplazar caracteres no deseados (como \xa0) y comas por puntos decimales
    clean_string = price_string.replace('\xa0', '').replace(',', '.')

    # Extraer los números de la cadena
    numbers = re.findall(r'\d+\.\d+', clean_string)

    # Convertir los números a valores decimales
    decimal_numbers = [float(number) for number in numbers]

    # Extraer la moneda de la cadena
    currency = re.findall(r'[A-Z]+[$]', clean_string)[0]
    
    # Convierto el signo de dolares
    if currency in ['U$S','US$','U$D']:
        currency = 'USD'
    
    # Calcular el promedio o devolver el único precio
    if len(numbers) == 1:
        # Si solo hay un número, usar ese número como promedio
        average_price = float(numbers[0])
    else:
        # Si hay dos números, calcular el promedio
        average_price = sum(decimal_numbers) / len(decimal_numbers)
        average_price = round( average_price, 3 )
    
    return average_price, currency

def extract_review_count(review_string):
    """
    Extrae el número de reseñas de una cadena con formato específico.
    
    Parámetros:
    review_string (str): Cadena que contiene la información de reseñas.
    
    Retorna:
    int: El número de reseñas.
    """
    # Usar una expresión regular para buscar el número de reseñas dentro de paréntesis
    match = re.search(r'\(\d+.*\)', review_string)
    
    # Si se encuentra una coincidencia, devolver el número encontrado como entero
    if match:
        match_str = match.group(0)[1:-1]
        rate_count = int(match_str.split()[0])
        return rate_count
    else:
        # Si no se encuentra ninguna coincidencia, devolver None o manejar el caso como se prefiera
        return None

In [100]:
# Defino la URL
topic = 'juguetes'
url = 'https://spanish.alibaba.com/trade/search?SearchText={}'.format(topic)
url

'https://spanish.alibaba.com/trade/search?SearchText=juguetes'

In [101]:
# Pido el contenido HTML
response = get_http_response(url)
response

URL [https://spanish.alibaba.com/trade/search?SearchText=juguetes], HTTP status [True], HTTP code [200]


 <!-- tangram:6640 begin-->
<!DOCTYPE html>

<html class="rwd">
<head>
<script>
        window.__BB = { 
          ...(window.__BB||{}),
          disableStore: true,
          autoReportAPI: true,
          autoReportPerf: true,
          mode: -1,
          token: "3c5d1b712654492388240c69a6fc9655",
          scene: location?.pathname?.match(/\/products\//)?.index === 0 || (document?.referrer === '' && !location?.href?.includes('spm=')) ? "pc-search-all-crawler" : "pc-search-all-high-performance",
          group: '@@_pcFindSimilar@@base@@_pcSsrRenderUpgrade@@@@_pcSplitRequest@@@@_searchProductsVersion@@0.1.128@@_p4pContact@@@@_pageNameSsr@@search-all-mini-ad@@_pcAdVideo302@@old@@_activeViewType@@L@@_resultSize@@48@@_18popUp@@false@@_pageName@@search-all-mini-ad@@_isBrandAd@@true@@_isStarBrandModel@@@@_isTopBoothModel@@true@@_topBoothModelId@@2000000117@@_isToprank@@true@@_serviceRtStatistics@@@@_templateConfigKey@@main-search-nosn@@_servicePerformaceTime@@{"spOnlyTime":394,"business

In [99]:
products = response.find_all('div', class_=[
    'm-gallery-product-item-v2',
    'searchx-offer-item',
    'fy23-search-card',
    'J-search-card-wrapper',
    'fy23-list-card'
])
len(products)

4

In [114]:
product = products[1]

url = product.find('a').get('href') # Estoy asumiendo que es el primer link que encuentro
product_id = url.split('/')[-1].split('.')[0]
product_name = product.find('h2').text
description = ''

price_str = product.find('div', class_='search-card-e-price-main').text
price, currency =  calculate_average(price_str)

installments = ''
ranking = 0
store = product.find('a', class_='search-card-e-company').text

rating_tag = product.find('span', class_='search-card-e-review')
if rating_tag:
    rating = rating_tag.text.split('/')[0]
    rate_count = extract_review_count(rating_tag.text)
else:
    rating = 0
    rate_count = 0
platform = 'Alibaba'
is_promoted = bool( product.find('div', class_='ads-main-search-component-title-icon') )
is_best_seller = 0

print('product_id:', product_id)
print('product_name:', product_name)
print('description:', description)
print('price:', price)
print('currency:', currency)
print('installments:', installments)
print('ranking:', ranking)
print('rating:', rating)
print('rate_count:', rate_count)
print('platform:', platform)
print('store:', store)
print('is_promoted:', is_promoted)
print('is_best_seller:', is_best_seller)
print('url:', url)

product_id: Interactive-1601015248450
product_name: Accesorios interactivos para perros, deportes al aire libre, masticar, juguetes para perros, OVNI volador, Bolas Mágicas, descompresas, platillo, Bola de disco para perros
description: 
price: 1.375
currency: USD
installments: 
ranking: 0
rating: 4.9
rate_count: 11
platform: Ebay
store: Hangzhou Pinyu Commerce Co., Ltd.
is_promoted: False
is_best_seller: 0
url: https://spanish.alibaba.com/p-detail/Interactive-1601015248450.html?s=p
